In [ ]:
# Last run with HEAD at bcd27cb

In [1]:
%reload_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [2]:
# relative to nplinker source dir
import sys
sys.path.append('..')

In [3]:
import os
import numpy

from nplinker import nplinker
from nplinker import logconfig
logconfig.LogConfig.setLogLevel('DEBUG') # 'INFO' is a lot less spammy
from nplinker.scoring import IOKR


['/home/grimur/miniconda2/envs/jupyter-py3/lib/python3.7/site-packages/cdk_pywrapper']
Server process already running: True


In [4]:
npl = nplinker.NPLinker('carnegie.toml')

12:04:59 [DEBUG] config.py:83, Parsing default config file: /home/grimur/.config/nplinker/nplinker.toml
12:04:59 [DEBUG] config.py:86, Loading user config carnegie.toml
12:04:59 [DEBUG] loader.py:70, DatasetLoader(/home/grimur/uist-data/nplinker_datasets/carnegie_nomibig_27112019/)


In [6]:
npl.load_data()

12:05:54 [DEBUG] nplinker.py:201, load_data (normal case, full load)
12:05:54 [INFO] loader.py:303, Loaded global strain IDs (162 total)
12:05:54 [INFO] loader.py:311, Loaded dataset strain IDs (169 total)
12:05:54 [DEBUG] loader.py:274, load_spectra(/home/grimur/uist-data/nplinker_datasets/carnegie_nomibig_27112019/spectra/specs_ms.mgf)
12:05:55 [INFO] metabolomics.py:230, load_spectra loaded 3107 molecules
12:05:55 [DEBUG] loader.py:279, load_edges(/home/grimur/uist-data/nplinker_datasets/carnegie_nomibig_27112019/networkedges_selfloop/455bd15e8bd645789792b347648be818..selfloop)
12:05:55 [DEBUG] metabolomics.py:457, loading edges file: /home/grimur/uist-data/nplinker_datasets/carnegie_nomibig_27112019/networkedges_selfloop/455bd15e8bd645789792b347648be818..selfloop [3107 spectra from MGF]
12:05:55 [DEBUG] loader.py:282, load_metadata(/home/grimur/uist-data/nplinker_datasets/carnegie_nomibig_27112019/clusterinfo_summary/6f4bf34641bf4bcf8711042bc8cb086f.tsv)
12:05:55 [INFO] metabolomic

True

In [7]:
interesting_bgcs = [
"KRD026.Scaffold_10.region001",
"KRD026.Scaffold_13.region001",
"KRD140.Scaffold_2.region002",
"KRD140.Scaffold_3.region001",
"KRD162.Scaffold_13.region003",
"KRD175.Scaffold_6.region001",
"KRD197.Scaffold_2.region002"
]

In [ ]:
from nplinker.scoring import misc

In [51]:
links_for_bgcs = {}

for bgc_id in interesting_bgcs:
    link_score_lines = []
    for gcf in npl.gcfs:
        if bgc_id in [x.name for x in gcf.bgcs]:
            for spectrum in npl.spectra:
                score = misc.metcalf_scoring(spectrum, gcf, strains=npl.strains, standardised=True)

                for mf in npl.molfams:
                    if spectrum in mf.spectra:
                        break

                # bgc_id,gcf_id,spectrum_id,#spec,#gcf,#overlap,#strains,score,identical,p_val
                strain_name = bgc_id.split('.')[0]
                overlap = len(set(spectrum.strains.keys()).intersection(gcf.strains))
                info_line = [strain_name, 
                             bgc_id, 
                             gcf.gcf_id, 
                             gcf.product_type, 
                             spectrum.id, 
                             len(spectrum.strains), 
                             len(gcf.strains), 
                             overlap, 
                             len(npl.strains), 
                             score]

                link_score_lines.append(info_line)
                
    link_score_lines.sort(key=lambda x: x[9], reverse=True)
    links_for_bgcs[bgc_id] = link_score_lines

In [56]:
links_with_score_count = []

for bgc_id, links in links_for_bgcs.items():
    scores = [x[9] for x in links]
    for i, line in enumerate(links):
        if i == 10:
            break
        current_score = line[9]
        current_score_count = scores.count(current_score)
        line.append(current_score_count)

        links_with_score_count.append(line)
        print(line)

['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 55, 1, 1, 1, 17, 4.000000000000001, 14, 14]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 2663, 2, 1, 1, 17, 4.000000000000001, 14, 14]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 2665, 2, 1, 1, 17, 4.000000000000001, 14, 14]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 2668, 4, 1, 1, 17, 4.000000000000001, 14, 14]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 2679, 3, 1, 1, 17, 4.000000000000001, 14, 14]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 2682, 3, 1, 1, 17, 4.000000000000001, 14, 14]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 2685, 4, 1, 1, 17, 4.000000000000001, 14, 14]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 2826, 3, 1, 1, 17, 4.000000000000001, 14, 14]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 2972, 2, 1, 1, 17, 4.000000000000001, 14, 14]
['KRD026', 'KRD026.Scaffold_10.region001', 11, 'NRPS', 2980, 2, 1, 1, 17, 4.000000000000001, 

In [57]:
import csv

In [58]:
with open('metcalf_interesting_links_20200212.csv', 'w') as f:
    w = csv.writer(f)
    w.writerow(["# strain_name", 
                 "bgc_id",
                 "gcf_id",
                 "product_type",
                 "spectrum_id",
                 "spectrum_strains)",
                 "gcf_strains",
                 "overlap",
                 "total_strains",
                 "score",
                 "score_count"])
    w.writerows(links_with_score_count)